In [323]:
import pandas as pd
import random
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
import numpy as np
import matplotlib.pyplot as plt
import PyComplexHeatmap as pch
import matplotlib
import seaborn as sns
import biom
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [309]:
from skbio.stats.ordination import pcoa
from skbio import DistanceMatrix
from phylodm import PhyloDM
import dendropy

In [7]:
tax = pd.read_csv("../Pretraining_data_profile/Data/taxmap_slv_ssu_ref_nr_138.2.txt", sep="\t", low_memory=False)

acc = []
for i in range(tax.shape[0]):
    temp = tax.iloc[i]
    acc.append(f"{temp[0]}.{temp[1]}.{temp[2]}")

tax = tax.loc[:, "path"].str.split(';', expand=True)
tax.index = acc
tax = tax.iloc[:, 0: 7]
tax.columns = ["taxonomy_1", "taxonomy_2", "taxonomy_3", "taxonomy_4", "taxonomy_5", "taxonomy_6", "taxonomy_7"]

In [8]:
tax.head()

,taxonomy_1,taxonomy_2,taxonomy_3,taxonomy_4,taxonomy_5,taxonomy_6,taxonomy_7
A16379.1.1485,Bacteria,Pseudomonadota,Gammaproteobacteria,Enterobacterales,Pasteurellaceae,Haemophilus,
A45315.1.1521,Bacteria,Bacillota,Bacilli,Bacillales,Salisediminibacteriaceae,Salipaludibacillus,
A61579.1.1437,Bacteria,Thermotogota,Thermotogae,Thermotogales,Fervidobacteriaceae,Thermopallium,
AAAA02020713.1.1297,Bacteria,Pseudomonadota,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Escherichia-Shigella,
AAAA02020714.1.1202,Bacteria,Pseudomonadota,Alphaproteobacteria,Sphingomonadales,Sphingomonadaceae,Sphingomonas,


In [621]:
# embedding_file = "../../data/social_niche_embedding_100.txt"
# embedding_vector = pd.read_csv(embedding_file, sep=" ", index_col=0, header=None, dtype={0:str})
# embedding_vector.drop("<unk>", inplace=True)

In [622]:
embedding_vector = pd.read_csv("../../data/Embedding_list/PCA_100.txt",
                          header=None, sep=" ", low_memory=False, index_col=0)
embedding_vector.drop("<unk>", inplace=True)

In [597]:
file_path = "Data/traits_precalculated.txt"
bugbase_data = pd.read_csv(file_path, sep = "\t", index_col = 0)
bugbase_data.head()

,Aerobic,Anaerobic,Contains_Mobile_Elements,Facultatively_Anaerobic,Forms_Biofilms,Gram_Negative,Gram_Positive,Potentially_Pathogenic,Stress_Tolerant
HL281973.2.1492,0,1,0.004310,0,0.0,1,0,0.007299,0.000000
HL281749.1.1478,0,0,0.012931,0,0.0,0,1,0.007299,0.016949
AF542231.1.1414,0,1,0.004310,0,0.0,0,1,0.007299,0.016949
AF542233.1.1411,0,1,0.004310,0,0.0,0,1,0.007299,0.016949
AJ006963.1.1453,0,1,0.004310,0,0.0,0,1,0.007299,0.016949


### Facultatively_Anaerobic - Anaerobic = Aerobic 

### Genus

In [139]:
fid = bugbase_data.index.values
Gram_Positive = bugbase_data.loc[bugbase_data.Gram_Positive == 1]
fa_a_ana = Gram_Positive.loc[:, ['Facultatively_Anaerobic', 'Aerobic', 'Anaerobic']]
fa_a_ana['genus'] = tax.loc[fid, 'taxonomy_6']
fa_a_ana = fa_a_ana.groupby('genus').sum() 

In [140]:
genus_counts =[]
for i in range(0, fa_a_ana.shape[0]):
    genus_counts.append(sum(fa_a_ana.iloc[i] > 0))

In [141]:
fa_a_ana.loc[np.array(genus_counts) == 3]

,Facultatively_Anaerobic,Aerobic,Anaerobic
genus,,,
Corynebacterium,1,3,1
Incertae Sedis,49,11,678


No valid genus under the category Gram_Negative

In [28]:
fid = bugbase_data.index.values
Gram_Positive = bugbase_data.loc[bugbase_data.Gram_Negative == 1]
fa_a_ana = Gram_Positive.loc[:, ['Facultatively_Anaerobic', 'Aerobic', 'Anaerobic']]
fa_a_ana['genus'] = tax.loc[fid, 'taxonomy_6']
fa_a_ana = fa_a_ana.groupby('genus').sum() 
genus_counts =[]
for i in range(0, fa_a_ana.shape[0]):
    genus_counts.append(sum(fa_a_ana.iloc[i] > 0))
fa_a_ana.loc[np.array(genus_counts) == 3]

,Facultatively_Anaerobic,Aerobic,Anaerobic
genus,,,
Incertae Sedis,1,21,136


No valid genus under the category Gram_Negative

### family

In [29]:
fid = bugbase_data.index.values
Gram_Positive = bugbase_data.loc[bugbase_data.Gram_Positive == 1]
fa_a_ana = Gram_Positive.loc[:, ['Facultatively_Anaerobic', 'Aerobic', 'Anaerobic']]
fa_a_ana['family'] = tax.loc[fid, 'taxonomy_5']
fa_a_ana = fa_a_ana.groupby('family').sum() 
family_counts =[]
for i in range(0, fa_a_ana.shape[0]):
    family_counts.append(sum(fa_a_ana.iloc[i] > 0))
fa_a_ana.loc[np.array(family_counts) == 3]

,Facultatively_Anaerobic,Aerobic,Anaerobic
family,,,
Corynebacteriaceae,1,4,1
Incertae Sedis,37,2,159


No valid genus under the category Gram_Negative

In [30]:
fid = bugbase_data.index.values
Gram_Positive = bugbase_data.loc[bugbase_data.Gram_Negative == 1]
fa_a_ana = Gram_Positive.loc[:, ['Facultatively_Anaerobic', 'Aerobic', 'Anaerobic']]
fa_a_ana['family'] = tax.loc[fid, 'taxonomy_5']
fa_a_ana = fa_a_ana.groupby('family').sum() 
family_counts =[]
for i in range(0, fa_a_ana.shape[0]):
    family_counts.append(sum(fa_a_ana.iloc[i] > 0))
fa_a_ana.loc[np.array(family_counts) == 3]

,Facultatively_Anaerobic,Aerobic,Anaerobic
family,,,


No valid genus under the category Gram_Negative

### order

In [159]:
fid = bugbase_data.index.values
Gram_Positive = bugbase_data.loc[bugbase_data.Gram_Positive == 1]
fa_a_ana = Gram_Positive.loc[:, ['Facultatively_Anaerobic', 'Aerobic', 'Anaerobic']]
fa_a_ana['order'] = tax.loc[fid, 'taxonomy_4']
fa_a_ana = fa_a_ana.groupby('order').sum() 

order_counts =[]
for i in range(0, fa_a_ana.shape[0]):
    order_counts.append(sum(fa_a_ana.iloc[i] > 0))
fa_a_ana.loc[np.array(order_counts) == 3]

,Facultatively_Anaerobic,Aerobic,Anaerobic
order,,,
Lactobacillales,79,22,17
Mycobacteriales,1,6,1


In [160]:
o__Lactobacillales = tax.loc[fid][tax.loc[fid, 'taxonomy_4'] == "Lactobacillales"]
o__Lactobacillales_emb = embedding_vector.loc[[i for i in o__Lactobacillales.index]]

In [161]:
o__Lactobacillales_bugbase = bugbase_data.loc[o__Lactobacillales.index, 
                                             ["Facultatively_Anaerobic", "Aerobic", "Anaerobic"]]
o__Lactobacillales_bugbase = o__Lactobacillales_bugbase.reset_index()
o__Lactobacillales_bugbase = pd.melt(o__Lactobacillales_bugbase, id_vars='index', var_name='Type')
o__Lactobacillales_bugbase = o__Lactobacillales_bugbase.loc[o__Lactobacillales_bugbase.values == 1]

In [162]:
o__Lactobacillales_fa = np.array(o__Lactobacillales_bugbase['index'][o__Lactobacillales_bugbase.Type == "Facultatively_Anaerobic"])
o__Lactobacillales_ae = np.array(o__Lactobacillales_bugbase['index'][o__Lactobacillales_bugbase.Type == "Aerobic"])
o__Lactobacillales_an = np.array(o__Lactobacillales_bugbase['index'][o__Lactobacillales_bugbase.Type == "Anaerobic"])

In [163]:
data = np.random.rand(o__Lactobacillales_fa.shape[0] * o__Lactobacillales_an.shape[0], 100)
df = pd.DataFrame(data)

In [164]:
n = 0
index_name = []
for i in o__Lactobacillales_fa:
    for j in o__Lactobacillales_an:
        df.iloc[n] = embedding_vector.loc[i] - embedding_vector.loc[j]
        index_name.append(f'{i}_{j}')
        n = n + 1
df.index = index_name

In [125]:
# fid = o__Lactobacillales.index.values
# res = cosine_similarity(df, embedding_vector.loc[fid])
# res = pd.DataFrame(data=res, columns=fid)
# upper_triangle = np.triu(res, k=1)
# upper_triangle = pd.DataFrame(data=upper_triangle, index=res.index, columns=res.columns)
# upper_triangle['id_1'] = upper_triangle.index.tolist()
# upper_triangle = pd.melt(upper_triangle, id_vars="id_1")
# upper_triangle = upper_triangle.loc[upper_triangle.value != 0]
# upper_triangle.columns = ["id_1", "fid", "cosine"]
# upper_triangle['Type'] = ["temp"] * upper_triangle.shape[0]
# bugbase_data_temp = bugbase_data.loc[upper_triangle.fid]
# upper_triangle.loc[bugbase_data_temp.Facultatively_Anaerobic.values == 1, 'Type'] = "Facultatively_Anaerobic"
# upper_triangle.loc[bugbase_data_temp.Anaerobic.values == 1, 'Type'] = "Anaerobic"
# upper_triangle.loc[bugbase_data_temp.Aerobic.values == 1, 'Type'] = "Aerobic"
# upper_triangle = upper_triangle.loc[upper_triangle['Type'] != "temp"]
# upper_triangle.to_csv("order_Facultatively_Anaerobic_Anaerobic_Aerobic.csv", index=False)

In [180]:
fid = o__Lactobacillales.index.values
res = cosine_similarity(df, embedding_vector.loc[fid])
res = np.mean(res, axis=0)
res = pd.DataFrame({'fid':fid, 'cosine':res})
res = res.set_index('fid')
res['Type'] = ["temp"] * res.shape[0]
bugbase_data_temp = bugbase_data.loc[fid]

res.loc[bugbase_data_temp.loc[bugbase_data_temp.Facultatively_Anaerobic == 1].index, 'Type'] = "Facultatively_Anaerobic"
res.loc[bugbase_data_temp.loc[bugbase_data_temp.Anaerobic == 1].index, 'Type'] = "Anaerobic"
res.loc[bugbase_data_temp.loc[bugbase_data_temp.Aerobic == 1].index, 'Type'] = "Aerobic"
res = res.loc[res['Type'] != "temp"]

In [182]:
res.to_csv("Data/order_Facultatively_Anaerobic_Anaerobic_Aerobic_phy.csv")

In [63]:
fid = bugbase_data.index.values
Gram_Positive = bugbase_data.loc[bugbase_data.Gram_Negative == 1]
fa_a_ana = Gram_Positive.loc[:, ['Facultatively_Anaerobic', 'Aerobic', 'Anaerobic']]
fa_a_ana['order'] = tax.loc[fid, 'taxonomy_4']
fa_a_ana = fa_a_ana.groupby('order').sum() 

order_counts =[]
for i in range(0, fa_a_ana.shape[0]):
    order_counts.append(sum(fa_a_ana.iloc[i] > 0))
fa_a_ana.loc[np.array(order_counts) == 3]

,Facultatively_Anaerobic,Aerobic,Anaerobic
order,,,


### all OTUs

### Bugbase

In [623]:
fa_a_ana_bugbase = bugbase_data[['Facultatively_Anaerobic', 'Aerobic', 'Anaerobic']]
fa_a_ana_bugbase = fa_a_ana_bugbase.reset_index()
fa_a_ana_bugbase = pd.melt(fa_a_ana_bugbase, id_vars='index', var_name='Type')
fa_a_ana_bugbase = fa_a_ana_bugbase.loc[fa_a_ana_bugbase.values == 1]

In [624]:
all_fa = np.array(fa_a_ana_bugbase['index'][fa_a_ana_bugbase.Type == "Facultatively_Anaerobic"])
all_ae = np.array(fa_a_ana_bugbase['index'][fa_a_ana_bugbase.Type == "Aerobic"])
all_an = np.array(fa_a_ana_bugbase['index'][fa_a_ana_bugbase.Type == "Anaerobic"])

In [625]:
data = np.random.rand(all_fa.shape[0] * all_an.shape[0], 100)
df = pd.DataFrame(data)

In [626]:
n = 0
index_name = []
for i in all_fa:
    for j in all_an:
        df.iloc[n] = embedding_vector.loc[i] - embedding_vector.loc[j]
        index_name.append(f'{i}_{j}')
        n = n + 1
df.index = index_name

In [627]:
fid = fa_a_ana_bugbase["index"].values
res = cosine_similarity(df, embedding_vector.loc[fid])
res = np.mean(res, axis=0)
res = pd.DataFrame({'fid':fid, 'cosine':res})
res = res.set_index('fid')
res['Type'] = ["temp"] * res.shape[0]
bugbase_data_temp = bugbase_data.loc[fid]

res.loc[bugbase_data_temp.loc[bugbase_data_temp.Facultatively_Anaerobic == 1].index, 'Type'] = "facultatively"
res.loc[bugbase_data_temp.loc[bugbase_data_temp.Anaerobic == 1].index, 'Type'] = "anaerobic"
res.loc[bugbase_data_temp.loc[bugbase_data_temp.Aerobic == 1].index, 'Type'] = "aerobic"
res = res.loc[res['Type'] != "temp"]

In [628]:
res.to_csv("Data/Bugbase_Facultatively_Anaerobic_Anaerobic_Aerobic_all_phy.csv")

In [629]:
res.head()

,cosine,Type
fid,,
U96412.1.1468,0.021098,facultatively
U92197.1.1541,0.082306,facultatively
AB030220.1.1486,0.116751,facultatively
AB009222.1.1414,0.131083,facultatively
Y18188.1.1489,0.125595,facultatively


### Traitar

In [630]:
traits = pd.read_csv("Data/trait_predcit.csv", index_col=0)
traits.replace(3, 1, inplace=True)

### Aerobe Facultative Anaerobe
traits['Oxygen_Preference'] = 0

traits.loc[traits['Aerobe'] == 1, 'Oxygen_Preference'] = 1
traits.loc[traits['Facultative'] == 1, 'Oxygen_Preference'] = 2
traits.loc[traits['Anaerobe'] == 1, 'Oxygen_Preference'] = 3

oxygen_cols = ['Aerobe', 'Facultative', 'Anaerobe']
traits.loc[traits[oxygen_cols].sum(axis=1) != 1, 'Oxygen_Preference'] = None

In [631]:
fid = traits.index.values

In [632]:
all_fa = np.array(traits.index.values[traits["Oxygen_Preference"] == 2])
all_an = np.array(traits.index.values[traits["Oxygen_Preference"] == 3])
all_ae = np.array(traits.index.values[traits["Oxygen_Preference"] == 1])

In [633]:
data = np.random.rand(all_fa.shape[0] * all_an.shape[0], 100)
df = pd.DataFrame(data)

In [634]:
n = 0
index_name = []
for i in all_fa:
    for j in all_an:
        df.iloc[n] = embedding_vector.loc[i] - embedding_vector.loc[j]
        index_name.append(f'{i}_{j}')
        n = n + 1
df.index = index_name

fid = traits.index.values
res = cosine_similarity(df, embedding_vector.loc[fid])
res = np.mean(res, axis=0)
res = pd.DataFrame({'fid':fid, 'cosine':res})
res = res.set_index('fid')
res['Type'] = ["temp"] * res.shape[0]

In [635]:
res.loc[fid[traits["Oxygen_Preference"] == 2], 'Type'] = "facultatively"
res.loc[fid[traits["Oxygen_Preference"] == 3], 'Type'] = "anaerobic"
res.loc[fid[traits["Oxygen_Preference"] == 1], 'Type'] = "aerobic"
res = res.loc[res['Type'] != "temp"]

In [636]:
res.to_csv("Data/Traitar_Facultatively_Anaerobic_Anaerobic_Aerobic_all_phy.csv")

### bac Dive

In [637]:
traits = pd.read_csv("Data/trait_16s_output_pick.tsv", 
                     sep="\t")
traits = traits[["16s_accession", "oxygen_preference"]]
traits = traits.dropna()

In [638]:
traits["oxygen_preference"].unique()

array(['anaerobe', 'aerobe', 'facultative_anaerobe'], dtype=object)

In [639]:
all_fa = np.array(traits['16s_accession'][traits["oxygen_preference"] == "facultative_anaerobe"])
all_an = np.array(traits['16s_accession'][traits["oxygen_preference"] == "anaerobe"])
all_ae = np.array(traits['16s_accession'][traits["oxygen_preference"] == "aerobe"])

In [640]:
data = np.random.rand(all_fa.shape[0] * all_an.shape[0], 100)
df = pd.DataFrame(data)

In [641]:
n = 0
index_name = []
for i in all_fa:
    for j in all_an:
        df.iloc[n] = embedding_vector.loc[i] - embedding_vector.loc[j]
        index_name.append(f'{i}_{j}')
        n = n + 1
df.index = index_name

fid = traits['16s_accession'].values
res = cosine_similarity(df, embedding_vector.loc[fid])
res = np.mean(res, axis=0)
res = pd.DataFrame({'fid':fid, 'cosine':res})
res = res.set_index('fid')
res['Type'] = ["temp"] * res.shape[0]

In [642]:
res.loc[fid[traits["oxygen_preference"] == "facultative_anaerobe"], 'Type'] = "facultatively"
res.loc[fid[traits["oxygen_preference"] == "anaerobe"], 'Type'] = "anaerobic"
res.loc[fid[traits["oxygen_preference"] == "aerobe"], 'Type'] = "aerobic"
res = res.loc[res['Type'] != "temp"]

In [643]:
res.to_csv("Data/bacdive_Facultatively_Anaerobic_Anaerobic_Aerobic_all_phy.csv")

### GP_Aerobic - Aerobic + Anaerobic = GP_Anaerobic

### Genus

In [41]:
fid = bugbase_data.index.values
fa_a_ana = bugbase_data.loc[:, ['Aerobic', 'Anaerobic']]
fa_a_ana['genus'] = tax.loc[fid, 'taxonomy_6']
fa_a_ana = fa_a_ana.groupby('genus').sum() 

genus_counts =[]
for i in range(0, fa_a_ana.shape[0]):
    genus_counts.append(sum(fa_a_ana.iloc[i] > 0))
fa_a_ana.loc[np.array(genus_counts) == 2]

,Aerobic,Anaerobic
genus,,
Campylobacter,8,4
Corynebacterium,3,1
Incertae Sedis,32,814


In [42]:
fid = bugbase_data.index.values
g_Corynebacterium = tax.loc[fid][tax.loc[fid, 'taxonomy_6'] == "Corynebacterium"]

In [43]:
g_Corynebacterium.shape

(11, 7)

In [44]:
sum(bugbase_data.loc[g_Corynebacterium.index.values].Gram_Positive == 1)

11

In [45]:
fid = bugbase_data.index.values
g_Campylobacter = tax.loc[fid][tax.loc[fid, 'taxonomy_6'] == "Campylobacter"]
print(g_Campylobacter.shape)
print(sum(bugbase_data.loc[g_Campylobacter.index.values].Gram_Positive == 1))

(12, 7)
0


### family

In [46]:
fid = bugbase_data.index.values
fa_a_ana = bugbase_data.loc[:, ['Aerobic', 'Anaerobic']]
fa_a_ana['family'] = tax.loc[fid, 'taxonomy_5']
fa_a_ana = fa_a_ana.groupby('family').sum() 

family_counts =[]
for i in range(0, fa_a_ana.shape[0]):
    family_counts.append(sum(fa_a_ana.iloc[i] > 0))
fa_a_ana.loc[np.array(family_counts) == 2]

,Aerobic,Anaerobic
family,,
Actinomycetaceae,30,4
Bifidobacteriaceae,1,29
Campylobacteraceae,8,4
Corynebacteriaceae,4,1
Eggerthellaceae,1,44
Incertae Sedis,2,161


In [47]:
fid = bugbase_data.index.values
f_Actinomycetaceae = tax.loc[fid][tax.loc[fid, 'taxonomy_5'] == "Actinomycetaceae"]
print(f_Actinomycetaceae.shape)
print(sum(bugbase_data.loc[f_Actinomycetaceae.index.values].Gram_Positive == 1))

(34, 7)
34


In [48]:
fid = bugbase_data.index.values
f_Campylobacteraceae = tax.loc[fid][tax.loc[fid, 'taxonomy_5'] == "Campylobacteraceae"]
print(f_Campylobacteraceae.shape)
print(sum(bugbase_data.loc[f_Campylobacteraceae.index.values].Gram_Positive == 1))

(12, 7)
0


### order

In [49]:
fid = bugbase_data.index.values
Gram_Positive = bugbase_data.loc[bugbase_data.Gram_Positive == 1]
fa_a_ana = Gram_Positive.loc[:, ['Aerobic', 'Anaerobic']]
fa_a_ana['order'] = tax.loc[fid, 'taxonomy_4']
fa_a_ana = fa_a_ana.groupby('order').sum() 

order_counts =[]
for i in range(0, fa_a_ana.shape[0]):
    order_counts.append(sum(fa_a_ana.iloc[i] > 0))
fa_a_ana.loc[np.array(order_counts) == 2]

,Aerobic,Anaerobic
order,,
Actinomycetales,31,4
Bifidobacteriales,1,29
Coriobacteriales,9,85
Lactobacillales,22,17
Mycobacteriales,6,1


In [50]:
fid = bugbase_data.index.values
o_Actinomycetales = tax.loc[fid][tax.loc[fid, 'taxonomy_4'] == "Actinomycetales"]
print(o_Actinomycetales.shape)
print(sum(bugbase_data.loc[o_Actinomycetales.index.values].Gram_Positive == 1))

(35, 7)
35


In [51]:
fid = bugbase_data.index.values
o_Coriobacteriales = tax.loc[fid][tax.loc[fid, 'taxonomy_4'] == "Coriobacteriales"]
print(o_Coriobacteriales.shape)
print(sum(bugbase_data.loc[o_Coriobacteriales.index.values].Gram_Positive == 1))

(95, 7)
95


In [52]:
fid = bugbase_data.index.values
o_Lactobacillales = tax.loc[fid][tax.loc[fid, 'taxonomy_4'] == "Lactobacillales"]
print(o_Lactobacillales.shape)
print(sum(bugbase_data.loc[o_Lactobacillales.index.values].Gram_Positive == 1))

(149, 7)
149
